My attempts at using tableau for the visualization were insightful but what I wanted to do proved to be out of the realm of possibility for tableau.  Not a big deal but an interesting and mildly frustrating experience but positive since I now have a greater understanding of tableau and its limitations and have an opportunity to dive deeper into python coding with Dash.

This is my first attempt at creating an 'app' that will take user inputted ingredients, the serving size--likely to be replaced with a new variable "measurement"--and calculates the nutritional value of the recipe the user provides.  It definitely needs to be cleaned in a way that makes reading it more manageable but it works at the base level.

In [1]:
import pandas as pd
import plotly.express as px

import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
from dash import dash_table


# Load Data
nutrition_data = pd.read_csv('cleaned_nutrition_data.csv')
#recipe_data = pd.read_excel('recipe_nutrition.xlsx', sheet_name = 'Recipe Inputs')

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1('Nutrition Calculator'),
    dcc.Dropdown(
        id = 'ingredient-dropdown',
        options = [{'label' : i, 'value': i} for i in nutrition_data['name'].unique()],
        multi = True,
        placeholder = 'Select Ingredients'
    ),
    html.Div(id = 'serving-size-inputs'),
    html.Button('Submit', id = 'submit-button'),
    html.Div(id = 'output-container')
])

@app.callback(
    Output('serving-size-inputs', 'children'),
    [Input('ingredient-dropdown', 'value')]
)

def update_serving_size_inputs(selected_ingredients):
    if not selected_ingredients:
        return []
    return [html.Div([
        html.Label(f'Serving size for {ingredient} (g):'),
        dcc.Input(id = {'type': 'serving-size', 'index': ingredient}, type = 'number', value = 0)
    ]) for ingredient in selected_ingredients]

@app.callback(
    Output('output-container', 'children'),
    [Input('submit-button', 'n_clicks')],
    [State('ingredient-dropdown', 'value'),
     State({'type': 'serving-size', 'index': dash.dependencies.ALL}, 'value')]
)

def update_output(n_clicks, selected_ingredients, serving_sizes):

    if not n_clicks or not selected_ingredients:
        return dash_table.DataTable(data=[], columns=[]), {}
    # Create a DataFrame with the selected ingredients and serving sizes
    selected_data= pd.DataFrame({'Ingredient' : selected_ingredients, 'Serving_Size': serving_sizes})
    
    # Merge with nutrition data to get nutritional values
    merged_data = pd.merge(selected_data, nutrition_data, left_on = 'Ingredient', right_on = 'name')

    # Calculate total values
    merged_data['Calories'] = merged_data['calories (kcal)'] * merged_data['Serving_Size'] / 100
    merged_data['Total Fats'] = merged_data['total_fat (g)'] * merged_data['Serving_Size'] / 100
    merged_data['Saturated Fats'] = merged_data['saturated_fat (g)'] * merged_data['Serving_Size'] / 100
    merged_data['Cholesterol'] = merged_data['cholesterol (mg)'] * merged_data['Serving_Size'] / 100
    merged_data['Carbohydrates'] = merged_data['carbohydrate (g)'] * merged_data['Serving_Size'] / 100
    merged_data['Sugars'] = merged_data['sugars (g)'] * merged_data['Serving_Size'] / 100
    merged_data['Fiber'] = merged_data['fiber (g)'] * merged_data['Serving_Size'] / 100
    
    # Create a list of the values and the associated DataFrame to be displayed by the app
    display_columns = ['Ingredient', 'Serving_Size', 'Calories', 'Total Fats', 'Saturated Fats', 'Cholesterol',
                        'Carbohydrates', 'Sugars', 'Fiber']
    display_data = merged_data[display_columns]

    # Calculate the Totals for each nutritional value displayed
    total_row = pd.DataFrame(display_data[['Calories', 'Total Fats', 'Saturated Fats', 'Cholesterol',
                        'Carbohydrates', 'Sugars', 'Fiber']].sum()).T
    total_row['Ingredient'] = 'Totals'
    total_row['Serving_Size'] = ''
    total_row = total_row.round(2)
    display_data = pd.concat([display_data, total_row])
    
    if display_data.empty:
        return dash_table.DataTable(), {}
    
    # Prepare data for plotting with nutrients on the x-axis and ingredients stacked
    melted_data = display_data.melt(id_vars=['Ingredient'], 
                                    value_vars=['Calories', 'Total Fats', 'Saturated Fats', 'Cholesterol', 
                                                'Carbohydrates', 'Sugars', 'Fiber'],
                                    var_name='Nutrient', value_name='Value')
    # Filter out the total row here
    melted_data = melted_data[melted_data['Ingredient'] != 'Totals']

    # Plot the data as stacked bar charts with nutrients on the x-axis and ingredients stacked
    fig = px.bar(
        melted_data,
        x='Nutrient',
        y='Value',
        color='Ingredient',  # Color the bars by ingredient
        title='Nutritional Contribution by Ingredient and Nutrient',
        labels={'Value': 'Nutritional Value', 'Nutrient': 'Nutrient'},
        barmode='stack'
    )

    # Display the DataFrame as a table
    table = dash_table.DataTable(
        columns = [{'name': i, 'id': i} for i in display_data.columns],
        data = display_data.to_dict('records')
    )
    return html.Div([
    dash_table.DataTable(
        columns=[{'name': i, 'id': i} for i in display_data.columns],
        data=display_data.to_dict('records')
    ),
    dcc.Graph(figure=fig)
])


if __name__ == '__main__':
    app.run_server(debug=True)

    

